In [68]:
import pandas as pd
import pdfplumber
import camelot

MAX_Y = 792
filename = '8.21.20_COVID-19_Update_FINAL.pdf'

with pdfplumber.open(filename) as pdf:

    for i, page in enumerate(pdf.pages):
        if "Project Roomkey Locations:" in page.extract_text():
            start_page = page
        if "Overall Totals:" in page.extract_text():
            end_page = page

    try:
        print("Start page:", start_page.page_number)
        print("End page:", end_page.page_number)
    except:
        print("Error: couldn't find project roomkey data :(")

    start_page_words = start_page.extract_words()
    words = list(map(lambda x: x.get('text'), start_page.extract_words()))

    for i, word in enumerate(start_page_words):
        if (words[i] == "Project" and words[i + 1] == "Roomkey" and words[i + 2] == "Locations:"):
            x_start = start_page_words[i + 3].get('x0') - 5
            y_start = MAX_Y - start_page_words[i + 3].get('top')

    y_page_end = MAX_Y - (start_page_words[-1].get('top') - 18)

    end_page_words = end_page.extract_words()
    words = list(map(lambda x: x.get('text'), end_page.extract_words()))

    y_page_start = MAX_Y - end_page_words[0].get('top')

    for i, word in enumerate(end_page_words):
        if (words[i] == "Overall" and words[i + 1] == "Totals:"):
            x_end = end_page_words[i + 4].get('x1') + 27
            y_end = MAX_Y - end_page_words[i + 4].get('bottom')

    labels = ["x_start", "x_end", "y_start", "y_end", "y_page_start", "y_page_end"]
    coords = dict(zip(labels, (x_start, x_end, y_start, y_end, y_page_start, y_page_end)))

    try:
        print(coords)
    except:
        print("Error: couldn't get table coords :(")



    """
    x of table beginning ("Locations:" + 1)
    x of table end ("Clients" + 1, "Overall Totals" + 3)
    y of table beginning ("Locations:" + 1)
    y of table end ("Overall Totals", "Overall Totals" + 3)
    
    y of new page beginning (first word)
    y of new page end (5th to last word)
    """

    all_data = []
    all_tables = []

    # For each page containing tables of data...
    for page_number in range(start_page.page_number, end_page.page_number + 1):
        page = pdf.pages[page_number - 1]

        # Get (x,y) coordinates for top left and bottom right corner of the table
        x0 = x_start
        y0 = y_start if page_number == start_page.page_number else y_page_start
        x1 = x_end
        y1 = y_end if page_number == end_page.page_number else y_page_end
    
        # Save a cropped image of the table
        cropped = page.crop((x0, MAX_Y - y0, x1, MAX_Y - y1))
        img = cropped.to_image()
        img.save(str(page_number) + '_' + filename[:-4] + '.png', format="PNG")

        # Read the table's data
        region = ','.join(str(coord) for coord in [x0, y0, x1, y1])
        table = camelot.read_pdf(filename, pages=str(page_number), table_regions=[region], strip_text='\n')
        all_tables.append(table[0].df)

    # Join tables from all pages
    df = pd.concat(all_tables)

    # Get columns
    df.columns = df.iloc[0]
    df = df.iloc[1:]

    # Drop empty columns if scan region is too wide
    if (df.iat[0, 0] == ''):
        df.drop(df.columns[0], axis=1, inplace=True)
    if (df.iat[0, -1] == ''):
        df.drop(df.columns[-1], axis=1, inplace=True)
        
    # Copy "Total Rooms" col over "Date Operational" col and rename it "# Operational"
    # Rename "Service Planning Area (SPA)" col to "SPA"
    df['Date Operational'] = df['Total Rooms']
    df.rename(columns={'Date Operational': '# Operational', 'Service Planning Area (SPA)': 'SPA'}, inplace=True)

    # Reformat SPA col cells: "SPA 1 – Antelope Valley (#1)" -> "Antelope Valley - 1"
    df['SPA'].replace({ '^SPA\s?\d+\s*–?\s*': '', '\(\D*(\d+)\)$': r'- \1' }, inplace=True, regex=True)

    # Export all table data
    df.to_csv('all_data.csv', index=False)



Start page: 4
End page: 6
{'x_start': Decimal('157.620'), 'x_end': Decimal('546.148'), 'y_start': Decimal('135.860'), 'y_end': Decimal('312.170'), 'y_page_start': Decimal('736.680'), 'y_page_end': Decimal('61.080')}
First time
1          SPA 1 – Antelope Valley (#1)
0                SPA 8 – South Bay (#1)
1     SPA 2 – San Fernando Valley (# 1)
2         SPA 3 San Gabriel Valley (#1)
3      SPA 2 – San Fernando Valley (#2)
4      SPA 2 – San Fernando Valley (#3)
5                     SPA 5 – West (#1)
6                     SPA 7 – East (#1)
7       SPA 3 – San Gabriel Valley (#2)
8                    SPA 4 – Metro (#1)
9                    SPA 4 – Metro (#2)
10      SPA 3 – San Gabriel Valley (#3)
11               SPA 8 – South Bay (#2)
12     SPA 2 – San Fernando Valley (#4)
13               SPA 8 – South Bay (#4)
14               SPA 8 – South Bay (#5)
15                   SPA 6 – South (#1)
16     SPA 2 – San Fernando Valley (#5)
17                    SPA 5 – West (#2)
18           

In [67]:
import re

def format_spa(spa):
    strip_spa = re.compile('^SPA\s?\d+\s*–?\s*')
    format_num = re.compile('\(\D*(\d+)\)$')

    stripped_spa = strip_spa.sub('', spa)
    formatted_num = format_num.sub(r'- \1', stripped_spa)

    return(formatted_num)
    


print(format_spa('SPA 1 – Antelope Valley (#1)'))
print(format_spa('SPA 3 San Gabriel Valley (#1)'))
print(format_spa('SPA 2 – San Fernando Valley (# 1)'))
print(format_spa('SPA 7– East (#3)'))
print(format_spa('SPA 6 –South (#2)'))

Antelope Valley - 1
San Gabriel Valley - 1
San Fernando Valley - 1
East - 3
South - 2
